In [1]:
import pandas as pd
from sqlalchemy import create_engine
import re
import json
import requests

In [2]:
engine_connection = create_engine('sqlite:///../data/raw/raw_data_project_m1.db')

In [3]:
my_tables=engine_connection.table_names()

for i in range(0,len(my_tables)):
    x=my_tables[i]
    query=f'SELECT * FROM {x}'
    df_i = pd.read_sql(query, con=engine_connection)
    if i==0:
        df_new=df_i
    else:
        df_new=pd.merge(df_new,df_i, how='left', on='uuid')
df_new

,uuid,dem_education_level,dem_full_time_job,normalized_job_code,country_code,rural,age,gender,dem_has_children,age_group,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None,AT,countryside,61 years old,male,NO,40_65,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0,AT,urban,57 years old,male,yES,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None,AT,city,32 years old,male,nO,26_39,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,AT,Country,45 years old,Male,YES,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,AT,city,41 years old,Fem,yES,40_65,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f,SK,urban,37 years old,FeMale,nO,26_39,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962,SK,urban,53 years old,Male,yES,40_65,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None,SK,Non-Rural,1992,male,NO,juvenile,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a,SK,city,47 years old,male,yES,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...


In [4]:
df_new.columns

Index(['uuid', 'dem_education_level', 'dem_full_time_job',
       'normalized_job_code', 'country_code', 'rural', 'age', 'gender',
       'dem_has_children', 'age_group',
       'question_bbi_2016wave4_basicincome_awareness',
       'question_bbi_2016wave4_basicincome_vote',
       'question_bbi_2016wave4_basicincome_effect',
       'question_bbi_2016wave4_basicincome_argumentsfor',
       'question_bbi_2016wave4_basicincome_argumentsagainst'],
      dtype='object')

In [5]:
df_new=df_new.drop_duplicates()

In [6]:
# the function to clean age column and transform into integer value
def clean_age(x):
    if (len(re.findall('[0-9]{4}',x))>0 and len(x)==4):
        return 2016-int(x)
    else:
        y=re.findall('[0-9]{2}',x)
        return int(y[0])
    
# Now I clean the age column
df_new['age']=df_new['age'].map(clean_age)

In [7]:
# the function to clean gender column and transform into (male, female)
def clean_gender(x):
    if (x=='Fem'):
        return 'female'
    else:
        return x.lower()

# Now I clean the age column
df_new['gender']=df_new['gender'].map(clean_gender)

In [8]:
# the function to clean gender column and transform into (male, female)
def clean_has_children(x):
    return x.lower()

# Now I clean the age column
df_new['dem_has_children']=df_new['dem_has_children'].map(clean_has_children)

In [9]:
# the function to clean gender column and transform into (male, female)
def clean_age_group(x):
    if (x=='juvenile'):
        return '14_25'
    else:
        return x

# Now I clean the age_group column
df_new['age_group']=df_new['age_group'].map(clean_age_group)

In [10]:
# the function to clean rural column and transform into (rural, non-rural)
def clean_rural(x):
    if (x=='urban' or x=='city'):
        return 'non_rural'
    elif x=='Country' or x=='countryside':
        return 'rural'
    else:
        return x.lower()

# Now I clean the rural colum
df_new['rural']=df_new['rural'].map(clean_rural)

In [11]:
# the function to clean country_code column and transform GB to UK and GR to EL
def clean_country_code(x):
    if (x=='GR'):
        return 'EL'
    elif x=='GB':
        return 'UK'
    else:
        return x

# Now I clean the rural colum
df_new['country_code']=df_new['country_code'].map(clean_country_code)

In [12]:
df_new


,uuid,dem_education_level,dem_full_time_job,normalized_job_code,country_code,rural,age,gender,dem_has_children,age_group,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None,AT,rural,61,male,no,40_65,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0,AT,non_rural,57,male,yes,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None,AT,non_rural,32,male,no,26_39,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,AT,rural,45,male,yes,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,AT,non_rural,41,female,yes,40_65,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f,SK,non_rural,37,female,no,26_39,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962,SK,non_rural,53,male,yes,40_65,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None,SK,non-rural,24,male,no,14_25,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a,SK,non_rural,47,male,yes,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...


In [13]:
############# API

#1st I create a list with uniq values of normalized_jod_code in the table
my_l=list(set(df_new['normalized_job_code'].to_list()))
len(my_l)
my_result=[]
for i in my_l:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
    results = response.json()
    my_result.append(results)
  
df_jobs=pd.DataFrame(my_result)
df_jobs=df_jobs.drop('error', axis=1)

In [14]:
df_jobs

,uuid,title,normalized_job_title,parent_uuid
0,76385cb24e831b3364279b9de5641d8f,Automatic Data Processing Systems Security Spe...,automatic data processing systems security spe...,7ef30a5b2e0ec04db4ad4d5b1d354c32
1,631bc3a12ed11340fc94ebfffa8f5f4e,Database Technician,database technician,0581dd6e7d2b00f3d449837e60ab76a0
2,36675c937946c8a4fc5de7d0b4c850de,Data Warehousing Engineer,data warehousing engineer,72ac7ec4ef0212a3aa54ed92abac8a92
3,f670246beda8187fbe875401e7bc4ba6,Data Processing Services Sales Representative,data processing services sales representative,96fe8c70b3c2f9ca197722fe88c5d6ca
4,122f24bc911a79c26ca66210557dda07,Clinical Data Specialist,clinical data specialist,db06efbaa9cb31d42e02c047ffb0a15a
...,...,...,...,...
152,fd69391dc29c83d5ad744daa57de0175,Data Transcriber,data transcriber,fcfdb76b27a6e4c34abd128cbd9f61ba
153,73d251075a710326ae436d9819246377,Databases Software Consultant,databases software consultant,b90ca4df5690002377a7b0f1f3d40781
154,2616d64becb4f2c478dd42f501bbfbff,Data Processing Control Clerk,data processing control clerk,9b44dbc6767274d8b3a820e133803d00
155,f2ecf36d7eb0a4225974fc6c4f48cfcb,Voice and Data Technician,voice and data technician,325e2251e20170928426473156bd3c48


In [15]:
df_jobs=df_jobs.rename(columns={'uuid':'normalized_job_code'})

In [16]:
################ web scraping

from bs4 import BeautifulSoup




In [17]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
ha=pd.read_html(url)
countries=dict()
now='C'
for df in ha:
    for f in range(0,len(df.index)):
        for c in range(0, len(df.columns)):
            if (isinstance(df.iloc[f,c],str)):
                if (now=='C'):
                    my_country=df.iloc[f,c]
                    now='CC'
                else:
                    my_country_code=df.iloc[f,c]
                    my_country_code=my_country_code.replace('(','')
                    my_country_code=my_country_code.replace(')','')
                    now='C'
                    countries[my_country]=my_country_code

countries           
     

{'Belgium': 'BE',
 'Greece': 'EL',
 'Lithuania': 'LT',
 'Portugal': 'PT',
 'Bulgaria': 'BG',
 'Spain': 'ES',
 'Luxembourg': 'LU',
 'Romania': 'RO',
 'Czechia': 'CZ',
 'France': 'FR',
 'Hungary': 'HU',
 'Slovenia': 'SI',
 'Denmark': 'DK',
 'Croatia': 'HR',
 'Malta': 'MT',
 'Slovakia': 'SK',
 'Germany': 'DE',
 'Italy': 'IT',
 'Netherlands': 'NL',
 'Finland': 'FI',
 'Estonia': 'EE',
 'Cyprus': 'CY',
 'Austria': 'AT',
 'Sweden': 'SE',
 'Ireland': 'IE',
 'Latvia': 'LV',
 'Poland': 'PL',
 'United Kingdom': 'UK',
 'Iceland': 'IS',
 'Norway': 'NO',
 'Liechtenstein': 'LI',
 'Switzerland': 'CH',
 'Montenegro': 'ME',
 'North Macedonia': 'MK',
 'Albania': 'AL',
 'Serbia': 'RS',
 'Turkey': 'TR',
 'Bosnia and Herzegovina': 'BA',
 'Kosovo*[1]': 'XK[2]',
 'Armenia': 'AM',
 'Belarus': 'BY',
 'Georgia': 'GE',
 'Azerbaijan': 'AZ',
 'Moldova': 'MD',
 'Ukraine': 'UA',
 'Algeria': 'DZ',
 'Lebanon': 'LB',
 'Syria': 'SY',
 'Egypt': 'EG',
 'Libya': 'LY',
 'Tunisia': 'TN',
 'Israel': 'IL',
 'Morocco': 'MA',
 'J

In [18]:
df_countries=pd.DataFrame(countries.items(), columns=['country', 'country_code'])
df_countries

,country,country_code
0,Belgium,BE
1,Greece,EL
2,Lithuania,LT
3,Portugal,PT
4,Bulgaria,BG
...,...,...
67,Taiwan,TW
68,Canada,CA
69,Japan,JP
70,Singapore,SG


In [19]:
df_total=pd.merge(df_new,df_jobs, how='left', on='normalized_job_code')
df_total

,uuid,dem_education_level,dem_full_time_job,normalized_job_code,country_code,rural,age,gender,dem_has_children,age_group,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,title,normalized_job_title,parent_uuid
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None,AT,rural,61,male,no,40_65,I know something about it,I would not vote,None of the above,None of the above,None of the above,NaN,NaN,NaN
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0,AT,non_rural,57,male,yes,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None,AT,non_rural,32,male,no,26_39,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,NaN,NaN,NaN
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,AT,rural,45,male,yes,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,Data Coordinator,data coordinator,0b9dd32a367f4562ec77b993053d1910
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,AT,non_rural,41,female,yes,40_65,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,Database Developer,database developer,b90ca4df5690002377a7b0f1f3d40781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f,SK,non_rural,37,female,no,26_39,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance,Data Warehouse Developer,data warehouse developer,72ac7ec4ef0212a3aa54ed92abac8a92
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962,SK,non_rural,53,male,yes,40_65,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...,Database Manager,database manager,b90ca4df5690002377a7b0f1f3d40781
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None,SK,non-rural,24,male,no,14_25,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above,NaN,NaN,NaN
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a,SK,non_rural,47,male,yes,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...,Data Officer,data officer,596226d5f9ef63de41a852826d483d19


In [20]:
df_total=pd.merge(df_total,df_countries,how='left',on='country_code')
df_total.to_csv('../data/processed/df_total.csv')
df_total

,uuid,dem_education_level,dem_full_time_job,normalized_job_code,country_code,rural,age,gender,dem_has_children,age_group,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,title,normalized_job_title,parent_uuid,country
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None,AT,rural,61,male,no,40_65,I know something about it,I would not vote,None of the above,None of the above,None of the above,NaN,NaN,NaN,Austria
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0,AT,non_rural,57,male,yes,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,Austria
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None,AT,non_rural,32,male,no,26_39,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,NaN,NaN,NaN,Austria
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,AT,rural,45,male,yes,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,Data Coordinator,data coordinator,0b9dd32a367f4562ec77b993053d1910,Austria
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,AT,non_rural,41,female,yes,40_65,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,Database Developer,database developer,b90ca4df5690002377a7b0f1f3d40781,Austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f,SK,non_rural,37,female,no,26_39,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance,Data Warehouse Developer,data warehouse developer,72ac7ec4ef0212a3aa54ed92abac8a92,Slovakia
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962,SK,non_rural,53,male,yes,40_65,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...,Database Manager,database manager,b90ca4df5690002377a7b0f1f3d40781,Slovakia
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None,SK,non-rural,24,male,no,14_25,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above,NaN,NaN,NaN,Slovakia
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a,SK,non_rural,47,male,yes,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...,Data Officer,data officer,596226d5f9ef63de41a852826d483d19,Slovakia


In [22]:
'Spain' in df_countries['country'].unique()

True

In [54]:
my_dict=df_total['question_bbi_2016wave4_basicincome_argumentsfor'].value_counts().to_dict()
my_dict

{'None of the above': 2163,
 'It reduces anxiety about financing basic needs': 1273,
 'It creates more equality of opportunity': 848,
 'It encourages financial independence and self-responsibility': 583,
 'It increases solidarity, because it is funded by everyone': 479,
 'It increases appreciation for household work and volunteering': 394,
 'It reduces bureaucracy and administrative expenses': 357,
 'It reduces anxiety about financing basic needs | It creates more equality of opportunity': 176,
 'It creates more equality of opportunity | It reduces anxiety about financing basic needs': 156,
 'It reduces anxiety about financing basic needs | It increases appreciation for household work and volunteering': 141,
 'It reduces anxiety about financing basic needs | It encourages financial independence and self-responsibility': 119,
 'It reduces anxiety about financing basic needs | It increases solidarity, because it is funded by everyone': 114,
 'It reduces anxiety about financing basic need

In [24]:
my_dict=df_total['question_bbi_2016wave4_basicincome_argumentsfor'].value_counts().to_dict()
list_arg=[]
list_qty=[]
for i, j in my_dict.items():
    l=i.split('|')
    for m in l:
        list_arg.append(m.strip())
        list_qty.append(j)
print(len(list_arg))
print(len(list_qty))
set(list_arg)
df_for=pd.DataFrame(list_arg, columns=['arguments_for'])
df_for['qty']=list_qty
df_for

2978
2978


,arguments_for,qty
0,None of the above,2163
1,It reduces anxiety about financing basic needs,1273
2,It creates more equality of opportunity,848
3,It encourages financial independence and self-...,583
4,"It increases solidarity, because it is funded ...",479
...,...,...
2973,It encourages financial independence and self-...,1
2974,It reduces anxiety about financing basic needs,1
2975,It increases appreciation for household work a...,1
2976,It reduces bureaucracy and administrative expe...,1


In [30]:
my_dict=df_total['question_bbi_2016wave4_basicincome_argumentsagainst'].value_counts().to_dict()
list_arg=[]
list_qty=[]
for i, j in my_dict.items():
    l=i.split('|')
    for m in l:
        list_arg.append(m.strip())
        list_qty.append(j)
print(len(list_arg))
print(len(list_qty))
set(list_arg)
df_against=pd.DataFrame(list_arg, columns=['arguments_against'])
df_against['qty']=list_qty
df_against

4010
4010


,arguments_against,qty
0,None of the above,1381
1,Only the people who need it most should get so...,875
2,It might encourage people to stop working,823
3,It is impossible to finance,767
4,Foreigners might come to my country and take a...,632
...,...,...
4005,It increases dependence on the state,1
4006,It might encourage people to stop working,1
4007,Only the people who need it most should get so...,1
4008,Foreigners might come to my country and take a...,1


In [26]:
 print(df_for['qty'].sum())
df_for[df_for['arguments_for']=='None of the above']['qty'].sum()

16734


2163

In [31]:
df_result=pd.DataFrame(data=None, index=['In Favor', 'Against'],
                                             columns=['Number of Pro Arguments','Number of Cons Arguments'])
df_result.iloc[1,0] = df_for[df_for['arguments_for']=='None of the above']['qty'].sum()
df_result.iloc[0,0] = df_for['qty'].sum() - df_for[df_for['arguments_for']=='None of the above']['qty'].sum()

df_result.iloc[1,1] = df_against[df_against['arguments_against']=='None of the above']['qty'].sum()
df_result.iloc[0,1] = df_against['qty'].sum() - df_against[df_against['arguments_against']=='None of the above']['qty'].sum()


In [32]:
df_result

,Number of Pro Arguments,Number of Cons Arguments
In Favor,14571,18374
Against,2163,1381


In [21]:


#test=df_new[df_new['country']=='Spain'].groupby(['country','title','rural']).size()
#test=df_new.groupby(['country','title','rural']).size()
#test=df_new.groupby(['uuid']).size()

#
#test = df_new.groupby(['country','title','rural']).agg({'title':'count'})
#test['title'].sum()
#test['percentage']=test['title'] / test['title'].sum()
#test.to_csv()

grouped=df_total[df_total['country']=='Bulgaria'].groupby(['country','title','rural']).size().reset_index()

#grouped=df_total.groupby(['country','title','rural']).size().reset_index()

#test=test.reset_index()
grouped = grouped.rename(columns={0: 'Quantity','country':'Country','title': 'Job Title', 'rural':'Rural'})
grouped['Percentage']=grouped['Quantity'] / grouped['Quantity'].sum() * 100
grouped


,Country,Job Title,Rural,Quantity,Percentage
0,Bulgaria,Automatic Data Processing Planner,non_rural,1,1.041667
1,Bulgaria,Automatic Data Processing Planner (ADP Planner),non_rural,1,1.041667
2,Bulgaria,Business Database Analyst,non_rural,1,1.041667
3,Bulgaria,Clinical Data Management Associate Director,non_rural,1,1.041667
4,Bulgaria,Clinical Data Manager,non-rural,1,1.041667
...,...,...,...,...,...
67,Bulgaria,Naval Tactical Data System--Combat Information...,non_rural,1,1.041667
68,Bulgaria,Naval Tactical Data System--Combat Information...,rural,1,1.041667
69,Bulgaria,SCADA Technician (Supervisory Control and Data...,non-rural,1,1.041667
70,Bulgaria,SQL Database Administrator,non_rural,2,2.083333


In [64]:
test.to_csv('../data/results/df_result_total.csv')



In [110]:
#df_new.loc[df_new['country'].isnull(),['country_code']]
df_new.isnull().sum()
#a=df_new['title'].isnull().sum()
#b=df_new['title'].value_counts().sum()
#print(a, b, a+b)
#df_new['dem_education_level'].value_counts()
#df_new.loc[(df_new['dem_education_level'].isnull()) & (~df_new['title'].isnull()),['title']]

uuid                                                      0
dem_education_level                                     663
dem_full_time_job                                         0
normalized_job_code                                    3947
country_code                                              0
rural                                                     0
age                                                       0
gender                                                    0
dem_has_children                                          0
age_group                                                 0
question_bbi_2016wave4_basicincome_awareness              0
question_bbi_2016wave4_basicincome_vote                   0
question_bbi_2016wave4_basicincome_effect                 0
question_bbi_2016wave4_basicincome_argumentsfor           0
question_bbi_2016wave4_basicincome_argumentsagainst       0
title                                                  3947
normalized_job_title                    

In [70]:
a='United States'
a.replace(' ','')

'UnitedStates'

In [33]:
############# API

#1st I create a list with uniq values of normalized_jod_code in the table
my_l=list(set(df_new['normalized_job_code'].to_list()))
len(my_l)
my_result=[]
for i in my_l:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}/related_skills')
    results = response.json()
    my_result.append(results)
  
df_skills=pd.DataFrame(my_result)


KeyError: "['error'] not found in axis"

In [35]:
response = requests.get('http://api.dataatwork.org/v1/jobs/a4d5b8b38f9513825d0d94a981ebe962/related_skills')
results = response.json()

In [ ]:
df_skills_new=pd.Dataframe(None, columns=['job_uuid','job_title','normalized_job_title','skill_uuid','skill_name','skill_type', 'description',
                                         'normalized_skill_name','importance','level'])
for i in range(0,len(df_skills.index)):
    df_skills_new['job_uuid'][i]=df_skills['job_uuid'][i]
    df_skills_new['job_title'][i]=df_skills['job_title'][i]
    df_skills_new['normalized_job_title'][i]=df_skills['normalized_job_title'][i]
    for j in df_skills['skills'][i]:  #es una lista de diccionarios
        j 
        

In [53]:
df_skills['job_uuid'][2]

nan

In [41]:
df_skills

,job_uuid,job_title,normalized_job_title,skills,error
0,76385cb24e831b3364279b9de5641d8f,Automatic Data Processing Systems Security Spe...,automatic data processing systems security spe...,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,NaN
1,631bc3a12ed11340fc94ebfffa8f5f4e,Database Technician,database technician,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,NaN
2,NaN,NaN,NaN,NaN,"{'code': 404, 'message': 'No associated skills..."
3,NaN,NaN,NaN,NaN,"{'code': 404, 'message': 'No associated skills..."
4,122f24bc911a79c26ca66210557dda07,Clinical Data Specialist,clinical data specialist,[{'skill_uuid': '162cb662c7ef4dc9e86a2b5e88cbc...,NaN
...,...,...,...,...,...
152,NaN,NaN,NaN,NaN,"{'code': 404, 'message': 'No associated skills..."
153,73d251075a710326ae436d9819246377,Databases Software Consultant,databases software consultant,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,NaN
154,2616d64becb4f2c478dd42f501bbfbff,Data Processing Control Clerk,data processing control clerk,[{'skill_uuid': 'e84312809bee7cb3fc8663def614f...,NaN
155,f2ecf36d7eb0a4225974fc6c4f48cfcb,Voice and Data Technician,voice and data technician,[{'skill_uuid': 'e72f8046dc4d704b6d1ca41dada93...,NaN


In [40]:
from pandas.io.json import json_normalize
df = pd.json_normalize(results)
df

,job_uuid,job_title,normalized_job_title,skills
0,a4d5b8b38f9513825d0d94a981ebe962,Database Manager,database manager,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...
